In [2]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.xgboost import Xgboost

In [3]:
td = TransfersDataset(Path.cwd()/'../prepped/final.csv')
td.encode_last_positions()
td.encode()
td.drop()

/Users/timurkambachekov/вышка/4 курс/вкр/project/ml/models/transfersdataset.py:7: DtypeWarning: Columns (38,59,75,88,92,103,149,170,186,199,203,214,260,281,297,310,314,325,371,392,408,421,425,436,704,725,741,754,758,769,815,836,852,865,869,880,926,947,963,976,980,991,1037,1058,1074,1087,1091,1102) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath_or_buffer=path, index_col=False).drop_duplicates().reset_index(drop=True)


Target propositions:

- Market Value at the end of the season (marketval_0)
- Difference between market values at the end of next and previous seasons (marketval_0 - marketval_-1)
- Age adjusted market value differences

# Target - Market Value

In [12]:
m = Xgboost(
    target = 'fee',
    features = ['age', 'season', 'window', 'loan', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo'] + \
               td.data.columns[td.data.columns.str.contains('pos')].tolist() + \
               td.data.columns[td.data.columns.str.contains('_-')].tolist(),
    data = td.data
)
m.train_test_split()
m.scale()
# m.tune_hp()
m.train()
m.inverse_scale()
m.mae()
m.mape()
m.plot_predictions()

MAE = 4346861.69322495
MAPE = 6.103886993109128e+21


In [5]:
m.top_n_predictions(20)

,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
547,W. Fofana,19,1.0,France,Ligue 1,Saint-Étienne,England,Premier League,Leicester City,1,35000000.0,0,40000000.0,10909535.00,29090465.00
1425,O. Zinchenko,25,3.0,England,Premier League,Manchester City,England,Premier League,Arsenal,1,35000000.0,0,42000000.0,17283800.00,24716200.00
541,Diogo Jota,23,1.0,England,Premier League,Wolverhampton Wanderers,England,Premier League,Liverpool,1,44700000.0,0,45000000.0,24611386.00,20388614.00
1293,Bruno Guimarães,24,2.0,France,Ligue 1,Olympique Lyonnais,England,Premier League,Newcastle United,0,42100000.0,0,40000000.0,20690110.00,19309890.00
1980,L. Openda,23,4.0,France,Ligue 1,Lens,Germany,Bundesliga,RB Leipzig,1,38500000.0,0,55000000.0,35887444.00,19112556.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,R. Ghezzal,27,0.0,England,Premier League,Leicester City,Italy,Serie A,Fiorentina,1,0.0,1,5500000.0,5635418.00,135418.00
1532,J. Clauss,29,3.0,France,Ligue 1,Lens,France,Ligue 1,Olympique Marseille,1,7500000.0,0,14000000.0,14086317.00,86317.00
242,J. Schmid,29,0.0,Germany,Bundesliga,Augsburg,Germany,Bundesliga,Freiburg,1,4000000.0,0,4000000.0,4062671.75,62671.75
2116,C. Okoli,22,4.0,Italy,Serie A,Atalanta,Italy,Serie A,Frosinone,1,100000.0,1,8000000.0,8042029.00,42029.00


In [6]:
m.top_n_predictions(20, worst=True)

,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
1629,C. Dagba,23,3.0,France,Ligue 1,Paris Saint-Germain,France,Ligue 1,Strasbourg,1,0.0,1,4500000.0,4489505.50,10494.50
2116,C. Okoli,22,4.0,Italy,Serie A,Atalanta,Italy,Serie A,Frosinone,1,100000.0,1,8000000.0,8042029.00,42029.00
242,J. Schmid,29,0.0,Germany,Bundesliga,Augsburg,Germany,Bundesliga,Freiburg,1,4000000.0,0,4000000.0,4062671.75,62671.75
1532,J. Clauss,29,3.0,France,Ligue 1,Lens,France,Ligue 1,Olympique Marseille,1,7500000.0,0,14000000.0,14086317.00,86317.00
207,R. Ghezzal,27,0.0,England,Premier League,Leicester City,Italy,Serie A,Fiorentina,1,0.0,1,5500000.0,5635418.00,135418.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,L. Openda,23,4.0,France,Ligue 1,Lens,Germany,Bundesliga,RB Leipzig,1,38500000.0,0,55000000.0,35887444.00,19112556.00
1293,Bruno Guimarães,24,2.0,France,Ligue 1,Olympique Lyonnais,England,Premier League,Newcastle United,0,42100000.0,0,40000000.0,20690110.00,19309890.00
541,Diogo Jota,23,1.0,England,Premier League,Wolverhampton Wanderers,England,Premier League,Liverpool,1,44700000.0,0,45000000.0,24611386.00,20388614.00
1425,O. Zinchenko,25,3.0,England,Premier League,Manchester City,England,Premier League,Arsenal,1,35000000.0,0,42000000.0,17283800.00,24716200.00


# Target - Market Value difference 

In [9]:
td.data['marketval_diff'] = td.data.marketval_0 - td.data['marketval_-1']
td.data['marketval_diff']

0              NaN
1      -34000000.0
2      -34000000.0
3        6000000.0
4              NaN
           ...    
2459           NaN
2460           NaN
2461           NaN
2462           NaN
2463           NaN
Name: marketval_diff, Length: 2464, dtype: float64

In [10]:
m = Xgboost(
    target = 'marketval_diff',
    features = ['age', 'season', 'window', 'loan', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo'] + \
               td.data.columns[td.data.columns.str.contains('pos')].tolist() + \
               td.data.columns[td.data.columns.str.contains('_-')].tolist(),
    data = td.data
)
m.train_test_split()
m.scale()
# m.tune_hp()
m.train()
m.inverse_scale()
m.mae()
m.mape()
m.plot_predictions()

MAE = 4631744.615364583
MAPE = 1.0977536312420823e+21


In [11]:
m.top_n_predictions(20, worst=True)

,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_diff,marketval_diff_pred,error
1909,K. Schlotterbeck,25,3.0,Germany,Bundesliga,Freiburg,Germany,Bundesliga,Bochum,0,0.0,1,0.0,3.007825e+04,3.007825e+04
2311,C. Lenz,28,4.0,Germany,Bundesliga,Eintracht Frankfurt,Germany,Bundesliga,RB Leipzig,1,1000000.0,0,-500000.0,-4.676788e+05,3.232125e+04
1186,X. Chavalerin,30,2.0,France,Ligue 1,Reims,France,Ligue 1,Troyes,1,1000000.0,0,-500000.0,-4.619374e+05,3.806256e+04
1297,L. Digne,28,2.0,England,Premier League,Everton,England,Premier League,Aston Villa,0,30000000.0,0,-10000000.0,-1.005089e+07,5.089400e+04
2116,C. Okoli,22,4.0,Italy,Serie A,Atalanta,Italy,Serie A,Frosinone,1,100000.0,1,-2000000.0,-1.918053e+06,8.194700e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,Diogo Jota,23,1.0,England,Premier League,Wolverhampton Wanderers,England,Premier League,Liverpool,1,44700000.0,0,17000000.0,-4.549458e+06,2.154946e+07
1425,O. Zinchenko,25,3.0,England,Premier League,Manchester City,England,Premier League,Arsenal,1,35000000.0,0,17000000.0,-4.871978e+06,2.187198e+07
1296,A. Martial,26,2.0,England,Premier League,Manchester United,Spain,LaLiga,Sevilla,0,0.0,1,-29000000.0,-5.714967e+06,2.328503e+07
1980,L. Openda,23,4.0,France,Ligue 1,Lens,Germany,Bundesliga,RB Leipzig,1,38500000.0,0,20000000.0,-1.197516e+07,3.197516e+07
